# 合并25个case的结果

In [1]:
import pandas as pd
from pathlib import Path
import re

# 1. 设置文件夹路a径
data_folder = Path(r"./data/")

# 2. 用正则表达式，从文件名中抓 outside 和 middle 的 setpoint
#    示例匹配：Outside_24_Middle_26_SDE2-03.csv
pattern = re.compile(r"Outside_(\d+)_Middle_(\d+)", re.IGNORECASE)

all_dfs = []

for csv_path in data_folder.glob("*.csv"):
    m = pattern.search(csv_path.name)
    if not m:
        print(f"⚠️ 文件名不符合模式，跳过：{csv_path.name}")
        continue
    
    outside_sp = int(m.group(1))   # Outside 的 setpoint 温度
    middle_sp  = int(m.group(2))   # Middle 的 setpoint 温度
    
    print(f"读取文件：{csv_path.name} | Outside={outside_sp}°C, Middle={middle_sp}°C")
    
    # 3. 读原始 CSV
    raw_df = pd.read_csv(csv_path)
    
    # 4. 丢掉前两行（单位 + 描述），只保留数值部分
    df = raw_df.iloc[2:].reset_index(drop=True)
    
    # 5. 转换数值列为 float（保留时间字符串列）
    for col in df.columns:
        if col == "Unnamed: 0":
            continue
        df[col] = pd.to_numeric(df[col], errors="coerce")
    
    # 6. 处理时间列，生成 datetime
    df["datetime"] = pd.to_datetime(
        "2024 " + df["Unnamed: 0"],
        format="%Y %d %b %H:%M"
    )
    df["hour"] = df["datetime"].dt.hour
    df["month"] = df["datetime"].dt.month
    
    # dayofyear 兼容不同 pandas 版本
    if hasattr(df["datetime"].dt, "dayofyear"):
        df["dayofyear"] = df["datetime"].dt.dayofyear
    else:
        df["dayofyear"] = df["datetime"].dt.day_of_year
    
    # 不再需要原始时间字符串列可以删掉
    df = df.drop(columns=["Unnamed: 0"])
    
    # 7. 把文件名中的 setpoint 信息加成两列
    df["Tset_outside"] = outside_sp
    df["Tset_middle"]  = middle_sp
    
    # 8. 记录一下数据来自哪个文件，方便以后 debug
    df["source_file"] = csv_path.name
    
    # 9. 放入列表等待拼接
    all_dfs.append(df)

# 10. 把所有 DataFrame 纵向拼接在一起
combined_df = pd.concat(all_dfs, ignore_index=True)

print("合并后的数据形状：", combined_df.shape)

# 11. 导出成一个总表 CSV
output_path = data_folder / "SDE2_all_cases_combined.csv"
combined_df.to_csv(output_path, index=False)

print("✅ 已保存整合文件到：", output_path)

读取文件：Outside_24_Middle_24_SDE2-03.csv | Outside=24°C, Middle=24°C
读取文件：Outside_24_Middle_25_SDE2-03.csv | Outside=24°C, Middle=25°C
读取文件：Outside_24_Middle_26_SDE2-03.csv | Outside=24°C, Middle=26°C
读取文件：Outside_24_Middle_27_SDE2-03.csv | Outside=24°C, Middle=27°C
读取文件：Outside_24_Middle_28_SDE2-03.csv | Outside=24°C, Middle=28°C
读取文件：Outside_25_Middle_24_SDE2-03.csv | Outside=25°C, Middle=24°C
读取文件：Outside_25_Middle_25_SDE2-03.csv | Outside=25°C, Middle=25°C
读取文件：Outside_25_Middle_26_SDE2-03.csv | Outside=25°C, Middle=26°C
读取文件：Outside_25_Middle_27_SDE2-03.csv | Outside=25°C, Middle=27°C
读取文件：Outside_25_Middle_28_SDE2-03.csv | Outside=25°C, Middle=28°C
读取文件：Outside_26_Middle_24_SDE2-03.csv | Outside=26°C, Middle=24°C
读取文件：Outside_26_Middle_25_SDE2-03.csv | Outside=26°C, Middle=25°C
读取文件：Outside_26_Middle_26_SDE2-03.csv | Outside=26°C, Middle=26°C
读取文件：Outside_26_Middle_27_SDE2-03.csv | Outside=26°C, Middle=27°C
读取文件：Outside_26_Middle_28_SDE2-03.csv | Outside=26°C, Middle=28°C
读取文件：Outsi

# energy分别是cooling、light和equipment的耗电量，总共14个房间，将他们分项加起来;COP设定为3，cooling列除以3得到HVAC

In [2]:
import pandas as pd
from pathlib import Path

# 1. 读取合并好的文件
data_folder = Path(r"./")
combined_path = data_folder / "SDE2_all_cases_combined.csv"
df_all = pd.read_csv(combined_path)

# 2. 找到所有 Energy 列
energy_cols = [c for c in df_all.columns if c.startswith("Energy")]
print("Energy 列数量：", len(energy_cols))

# 3. 按固定列号分段（这里假设你的 energy_cols 顺序和 CSV 文件一致）
# 注意：Python 切片左闭右开
cooling_cols   = energy_cols[0:14]   # 第7列 到 第14列
light_cols     = energy_cols[14:28]  # 第15列 到 第21列
equip_cols     = energy_cols[28:42]  # 第22列 到 第28列

print("Cooling columns:", cooling_cols)
print("Lighting columns:", light_cols)
print("Equipment columns:", equip_cols)

# 4. 计算分组能耗
df_all["E_cooling"]   = df_all[cooling_cols].sum(axis=1)
df_all["E_light"]     = df_all[light_cols].sum(axis=1)
df_all["E_equipment"] = df_all[equip_cols].sum(axis=1)

# 5. 计算建筑真实用电（考虑COP）
COP = 3.0
df_all["E_HVAC"] = df_all["E_cooling"] / COP
df_all["E_building_real"] = df_all["E_cooling"] / COP + df_all["E_light"] + df_all["E_equipment"]

# 6. 查看结果
df_all[["E_cooling", "E_light", "E_equipment", "E_building_real","E_HVAC"]].head()



Energy 列数量： 42
Cooling columns: ['Energy', 'Energy.1', 'Energy.2', 'Energy.3', 'Energy.4', 'Energy.5', 'Energy.6', 'Energy.7', 'Energy.8', 'Energy.9', 'Energy.10', 'Energy.11', 'Energy.12', 'Energy.13']
Lighting columns: ['Energy.14', 'Energy.15', 'Energy.16', 'Energy.17', 'Energy.18', 'Energy.19', 'Energy.20', 'Energy.21', 'Energy.22', 'Energy.23', 'Energy.24', 'Energy.25', 'Energy.26', 'Energy.27']
Equipment columns: ['Energy.28', 'Energy.29', 'Energy.30', 'Energy.31', 'Energy.32', 'Energy.33', 'Energy.34', 'Energy.35', 'Energy.36', 'Energy.37', 'Energy.38', 'Energy.39', 'Energy.40', 'Energy.41']


,E_cooling,E_light,E_equipment,E_building_real,E_HVAC
0,76.380952,0.847851,5.598523,31.906692,25.460317
1,75.514885,0.847851,5.598523,31.618003,25.171628
2,77.930653,0.847851,5.598523,32.423259,25.976884
3,79.302072,0.847851,5.598523,32.880398,26.434024
4,73.933641,0.847851,5.598523,31.090921,24.644547


# 处理数据集，先加上滞后的气象参数，前一小时前二小时

In [3]:
import pandas as pd

df = df_all.copy()

# 确保按 case + 时间排序
df = df.sort_values(["source_file", "datetime"])

# 定义要做滞后的气象列
weather_cols = [
    "Dry Bulb Temperature",
    "Relative Humidity",
    "Diffuse Horizontal Radiation"
]

# 对每个 case 单独做 shift，防止跨文件串联
group_cols = ["source_file"]  # 如果你还有别的 case 标识，也可以加上

for col in weather_cols:
    df[f"{col}_lag1"] = df.groupby(group_cols)[col].shift(1)
    df[f"{col}_lag2"] = df.groupby(group_cols)[col].shift(2)


df_model = df.dropna(subset=[f"{c}_lag1" for c in weather_cols] +
                             [f"{c}_lag2" for c in weather_cols])

df_model.head(10)



,Dry Bulb Temperature,Dew Point Temperature,Relative Humidity,Diffuse Horizontal Radiation,Energy,Energy.1,Energy.2,Energy.3,Energy.4,Energy.5,...,E_light,E_equipment,E_HVAC,E_building_real,Dry Bulb Temperature_lag1,Dry Bulb Temperature_lag2,Relative Humidity_lag1,Relative Humidity_lag2,Diffuse Horizontal Radiation_lag1,Diffuse Horizontal Radiation_lag2
2,25.6,24.7,95,0,8.119193,7.303527,11.028523,2.515576,3.542057,5.693886,...,0.847851,5.598523,25.976884,32.423259,25.0,25.0,100.0,94.0,0.0,0.0
3,26.0,25.0,94,0,8.331287,7.146449,11.354523,2.591383,3.671886,5.863833,...,0.847851,5.598523,26.434024,32.880398,25.6,25.0,95.0,100.0,0.0,0.0
4,26.0,25.0,94,0,7.945468,6.353466,10.785413,2.480894,3.512707,5.577643,...,0.847851,5.598523,24.644547,31.090921,26.0,25.6,94.0,95.0,0.0,0.0
5,25.3,24.9,98,0,7.373752,5.398101,9.948663,2.310844,3.262461,5.139871,...,0.767815,5.598523,22.315666,28.682004,26.0,26.0,94.0,94.0,0.0,0.0
6,25.0,25.0,100,0,7.173191,4.787212,9.699169,2.256957,3.192528,4.987545,...,0.767815,5.598523,21.756969,28.123307,25.3,26.0,98.0,94.0,0.0,0.0
7,25.0,25.0,100,37,7.153628,4.831854,9.776056,2.236646,3.188617,4.974224,...,0.767815,5.598523,22.023342,28.389680,25.0,25.3,100.0,98.0,0.0,0.0
8,25.1,25.0,99,84,7.260993,5.523300,10.033338,2.244005,3.226712,5.097194,...,0.767815,5.598523,23.228245,29.594583,25.0,25.0,100.0,100.0,37.0,0.0
9,25.0,25.0,100,157,7.427658,6.987193,10.121978,2.176062,3.180873,5.364547,...,0.767815,5.598523,24.823881,31.190219,25.1,25.0,99.0,100.0,84.0,37.0
10,25.0,24.0,94,239,7.842495,9.185048,10.254378,2.111057,3.109840,5.887602,...,0.767815,5.598523,27.472859,33.839198,25.0,25.1,100.0,99.0,157.0,84.0
11,24.5,24.2,98,272,8.541471,11.713357,10.358569,2.072003,3.047796,6.532520,...,0.767815,5.598523,30.126887,36.493225,25.0,25.0,94.0,100.0,239.0,157.0


In [5]:
output_path = data_folder / "preprocess.csv"
df_model.to_csv(output_path, index=False)
print("✅ 已保存：", output_path)

✅ 已保存： preprocess.csv
